<a href="https://colab.research.google.com/github/singularity014/BERT_FakeNews_Detection_Challenge/blob/master/NLP_NEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## I - Problem Statement

- **Given** - A Data Set of Fake and Real news.

- **Objective** - To develop a solution which  
  detects if a given news is Fake or Real.

- **Methodology used** - We try to pose the    problem as a text classification problem and build a deep learning model for achieving the objective.

#### Different Models - 
While there multiple types of models which could be used for building the solution for Text Classification. Some Examples are - 


*   **1D- Conv Net** - 
Yes, CNNs could also be used for text. **Advantage**: They are faster to ttrain. In fact given proper label, a CNN model could achieve decent accuracy **Disdvantage**: They fail to capture long term dependencies in Text, and doesnt not capture sequential information in text.

*   **RNN based models (LSTM, GRU) ** -  **Advantage** : They focus on considering words T each time step, encoding them by some non-linear calculation and then taking a decision. In simple words, they can capture sequential nature of a Text. 
Disadvantage - Slower to train, focus is more on sequential nature, and less on attention mechanism.

* **Transformer based Models(BERT, GPT2)**- 
Transformer based models are a breaktrhough in NLP, and tend to model. Which leverages multiple Transformer units, and a multi-headed attention mechanism. The advantage is that they focus only on attention mechanism. Thus we obtain a model, which is can used in context heavy applications.




## II - STandAlone BERT Model -  

- For our solution we will be using BERT model to develop Fake News or Real News Classification Solution. 

- We achieved an accuracy of 95+ % on test set, and a remarkable AUC by a standalone BERT Model. More improvements could be done with better tuning, and training for longer time. In cloud settings like Google Cloud(with larger GPUs) or AWS infra. But improvement is a continuous process :)

- We build an MVP with BERT Stand-alone model

- We can Also use, BERT (as an embedder) + LSTM model to build this solution. 

![alt text](https://lh3.googleusercontent.com/proxy/P18CTUGzSZSRWahZaJ-o3396dcWZZ7-CKWiTSM0oHf6FR7m_LWWKjRZ6hIArUKegAgd1e2zd01q7gySzmBMoEaK9V7A4TMUckvtRf0cSq6oukgE)

Above figure shows the Kind of model we will be building in developing our solution.




## III - Coding Environment  

- I chose Google Colab to document the process step by step, other than this I wanted to leverage the free GPU available in Google Colab.
- It helped me train faster, compared to my own PC.
- In production as well, GPU powered AWS frameweorks such as AWS Sagemkaer and Google GPU cloud infra are useful in training models and deploying quicker. 
- For coding environment we can develop models
  using Keras or Tensorflow. Depending upon    
  the level of control we want on model 
  creation, we can work with that TF versions. 

- For more control and inner workings of     
  model, TF core comes is usually very useful, 
  and for quicker prototyping we can go for  
  Keras or  TF2.0 which is keras style.

- For current solution, we can choose any of   
  the environment mentioned above, but I chose 
  to go with TF1.x for better control and   
  develop BERT TF.2.0 model for classification.



In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


## IV - Dependencies and BERT packages


*   We will mount the Google Drive to colab environment.
*   We will install BERT dependencies.
*   Import the necessary packages for developing model.



In [3]:
# Mounting Drive
from google.colab import drive 
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
# Installing BERT module
!pip install bert-tensorflow
# !pip install bert-for-tf2
# !pip install sentencepiece

     |████████████████████████████████| 71kB 3.1MB/s 


In [5]:
# Import packages
#------------ Utilities ---------
import numpy as np
import pandas as pd
import json
from sklearn.utils import shuffle
import nltk

# ---------- Tensorflow dependencies--------------------
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
import tensorflow_hub as hub
import tensorflow as tf

# ---------- BERT ------------
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [6]:
df_fktest=pd.read_json('https://storage.googleapis.com/public-resources/dataset/fake_test.json')
df_fktrain=pd.read_json('https://storage.googleapis.com/public-resources/dataset/fake_train.json')
df_realtest=pd.read_json('https://storage.googleapis.com/public-resources/dataset/real_test.json')
df_realtrain=pd.read_json('https://storage.googleapis.com/public-resources/dataset/real_train.json')

In [7]:
df_realtrain2=df_realtrain
df_realtest2=df_realtest
df_fktrain2=df_fktrain
df_fktest2=df_fktest

In [8]:
print(df_fktest.shape)
print(df_fktrain.shape)
print(df_realtest.shape)
print(df_realtrain.shape)

(200, 3)
(800, 3)
(200, 3)
(800, 3)


In [9]:
import re
def clean_txt(text):
    text = re.sub("'", "",text)
    text=re.sub("(\\W)+"," ",text)    
    return text

In [11]:
df_fktrain['text']  = df_fktrain.text.apply(clean_txt)
df_fktest['text']  = df_fktest.text.apply(clean_txt)
df_realtest['text']  = df_realtest.text.apply(clean_txt)
df_realtrain['text']  = df_realtrain.text.apply(clean_txt)

In [12]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//150 >0:
        n = len(text1.split())//150
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:200]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*150:w*150 + 200]
            l_total.append(" ".join(l_parcial))
    return l_total

In [13]:
df_fktest['text_split'] = df_fktest['text'].apply(get_split)
df_fktrain['text_split'] = df_fktrain['text'].apply(get_split)
df_realtest['text_split'] = df_realtest['text'].apply(get_split)
df_realtrain['text_split'] = df_realtrain['text'].apply(get_split)

In [14]:
print(df_fktest['text_split'].isnull().sum())
print(df_fktrain['text_split'].isnull().sum())
print(df_realtest['text_split'].isnull().sum())
print(df_realtrain['text_split'].isnull().sum())

0
0
0
0


In [15]:
df_fktrain['is_fake'] = 1
df_fktest['is_fake'] = 1
df_realtest['is_fake'] = 0
df_realtrain['is_fake'] = 0

In [16]:
train_data = pd.concat([df_fktrain, df_realtrain])
test_data = pd.concat([df_fktest, df_realtest])



# Shuffle the data
train_data = shuffle(train_data).reset_index(drop=True)
train_data.head(100)

,url,title,text,text_split,is_fake
0,https://arabic.rt.com/videoclub/1094755-%D8%B1...,Russia repelled the most dangerous epidemic of...,Russia repelled the most dangerous epidemic of...,[Russia repelled the most dangerous epidemic o...,1
1,https://www.theguardian.com/world/2020/feb/25/...,Iran deputy health minister : I have coronavirus,Irans deputy health minister I have coronaviru...,[Irans deputy health minister I have coronavir...,0
2,https://lv.sputniknews.ru/Latvia/20200315/1337...,Coronavirus invented in Latvia? Why not,Of course the application of the old Roman rul...,[Of course the application of the old Roman ru...,1
3,https://www.theatlantic.com/health/archive/202...,Why COVID - 19 Makes Some People Sicker Than O...,The COVID 19 crash comes suddenly In early Mar...,[The COVID 19 crash comes suddenly In early Ma...,0
4,https://www.khabarlb.com/world/khabar102875.html,The mystery of the virus that slept 5 years .....,Russia Today has published an article about t...,[Russia Today has published an article about t...,1
...,...,...,...,...,...
95,https://arabic.sputniknews.com/radio_between_t...,What is the truth behind the existence of a bi...,In response US Secretary of State Mike Pompeo ...,[In response US Secretary of State Mike Pompeo...,1
96,https://news.err.ee/1071565/reinsalu-thanks-uk...,Reinsalu thanks united kingdom and Belarus for...,Reinsalu thanks UK and Belarus for helping Est...,[Reinsalu thanks UK and Belarus for helping Es...,0
97,https://hathalyoum.net/articles/2292576-%D9%84...,Coronary virus puzzle that slept 5 years!,Coronary virus puzzle that slept 5 years This ...,[Coronary virus puzzle that slept 5 years This...,1
98,https://sudanway.com/2020/02/03/%D8%A7%D9%84%D...,Yellow skin is the host environment of the vir...,In a report published by the American Forbes m...,[In a report published by the American Forbes ...,1


In [17]:
train_l = []
label_l = []
index_l =[]
for idx,row in train_data.iterrows():
    for l in row['text_split']:
        train_l.append(l)
        label_l.append(row['is_fake'])
        index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(5481, 5481, 5481)

In [18]:
test_l = []
label_l2 = []
index_l2 =[]
for idx,row in test_data.iterrows():
    for l in row['text_split']:
        test_l.append(l)
        label_l2.append(row['is_fake'])
        index_l2.append(idx)
len(test_l), len(label_l2), len(index_l2)

(1230, 1230, 1230)

In [19]:
train_df = pd.DataFrame({'text':train_l, 'label':label_l})

In [20]:
test_df = pd.DataFrame({'text':test_l, 'label':label_l2})

In [21]:

train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x['text'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

In [22]:
test_InputExamples = test_df.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x['text'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

In [23]:
 
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"



def create_tokenizer_from_hub_module():

  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [24]:
label_list = [x for x in np.unique(train_data.is_fake)]

In [25]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 5481


INFO:tensorflow:Writing example 0 of 5481


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [ ]:

def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg,
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs, output_layer) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels,
          'pooled_output': output_layer
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [ ]:
OUTPUT_DIR = '/bert_news_category'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

In [ ]:
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [ ]:
from datetime import datetime
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=test_input_fn, steps=None)